In [ ]:
import sys
sys.path.append("../") # go to parent dir

from codebase.utils import MongoConnector
import pandas as pd
from fuzzywuzzy import process
from fuzzywuzzy import fuzz
from tqdm.notebook import tqdm

In [ ]:
conn = MongoConnector("../config.ini")

# Step1, Retrieve hashtags from Mongo

Take following times from Mongo:
> CPU times: user 9min 9s, sys: 11.7 s, total: 9min 20s  
> Wall time: 13min 26s

I have save them as pickle. Just read back them.

In [ ]:
# %%time

# Weeks = ["FirstWeek_March", "SecondWeek_March", "ThirdWeek_March", "FourthWeek_March",\
#          "FirstWeek_April", "SecondWeek_April", "ThirdWeek_April", "FourthWeek_April"]

# query = {}
# query["hashtags"] = {
#     u"$exists": True
# }

# rows_list = []
# for each_week in Weeks:
#     conn.get_collection_cursor(each_week)
#     docs = conn.data_streaming_from_collection(query=query)
#     for i, doc in enumerate(docs):
#         if i % 100000 == 0:
#             print(f"{each_week}: {i}")
        
#         row_dict = dict()
#         created_at = doc["created_at"]
#         for tag in doc["hashtags"]:
#             row_dict.update(created_at=created_at, hashtag=tag)
#             rows_list.append(row_dict)

# tagsDF = pd.DataFrame(rows_list)

In [ ]:
# save pickle
# tagsDF.to_pickle(path="./export/Hashtags-of-Mar-Apr.pkl.gz",
#                  compression="gzip")

# read pickle
tagsDF = pd.read_pickle("../export/Hashtags-of-Mar-Apr.pkl.gz",
                  compression="gzip")
tagsDF.reset_index()
tagsDF = tagsDF.set_index("created_at")

In [ ]:
tagsDF["hashtag"] = tagsDF["hashtag"].apply(str.lower)

# Apply Fuzzy Mapping (Optional)

In [ ]:
tagSet = set(tagsDF.hashtag.to_list())

In [ ]:
%%time
tagDict = {}
tagCandidates = set()
editing_threshold = 90

pbar = tqdm(total=len(tagSet))
for tag in tagSet:
    pbar.update(1)
    # first case
    if len(tagCandidates) == 0:
        tagDict[tag] = tag
        tagCandidates.add(tag)
        continue 
    
    most_sim_tag, sim = process.extractOne(tag, tagCandidates, scorer=fuzz.ratio)
#     print(f"{tag}: {most_sim_tag} {sim}")
    if sim >= editing_threshold:
        tagDict[tag] = most_sim_tag
    else:
        tagDict[tag] = tag
        tagCandidates.add(tag)
pbar.close()

In [ ]:
tagDict

# Step2, Resample counts of hashtag by week (or any time granuality)

> CPU times: user 21min 44s, sys: 3.19 s, total: 21min 47s  
> Wall time: 21min 46s

> CPU times: user 18min 16s, sys: 5.52 s, total: 18min 22s  
> Wall time: 18min 21s

In [ ]:
%%time
tagsDF = tagsDF.loc['2020-03-01':'2020-04-26']
grouped_tagsDF = tagsDF.groupby('hashtag').resample('W')['hashtag'].count()

In [ ]:
# grouped_tagsDF.to_pickle("./export/Counts-Hashtags-of-Mar-Apr.pkl.gz",
#                   compression="gzip")
# grouped_tagsDF = pd.read_pickle("./export/Counts-Hashtags-of-Mar-Apr.pkl.gz",
#                    compression="gzip")
# grouped_tagsDF.to_pickle("./export/Counts-lower-Hashtags-of-Mar-Apr.pkl.gz",
#                   compression="gzip")
grouped_tagsDF = pd.read_pickle("../export/Counts-lower-Hashtags-of-Mar-Apr.pkl.gz",
                   compression="gzip")

In [ ]:
gdf = grouped_tagsDF.unstack('hashtag', fill_value=0)

In [ ]:
gdf.shape

In [ ]:
gdf[["suicideprevention", "zerosuicide", "everylifematters"]].plot()

# Step3, Calculate count difference by week

In [ ]:
WeekDiff = gdf.diff(axis=0)

In [ ]:
WeekDiff.index

# Step4, Pick those hashtag with large count difference (Most Surging/Most Diminishing)

In [ ]:
# change to [::-1] to get descending results
# and [::1] to get ascending results
sort_ids = WeekDiff.iloc[5].argsort()[::-1][:50]

In [ ]:
WeekDiff.iloc[5][sort_ids]